## 常用调试流程
以下是可以参考的调试流程(面向新手):

- 如果网页有跳转，必须勾选 preservelog 防止丢包
- 看一下有没有框架 右键查看框架源代码(弹出式登陆界面)
- 登陆尽量使用错误密码 防止跳转
- 查看关键登陆包 分析哪些参数是加密的
- 使用别的浏览器分析哪些参数是固定的值
- 初步猜测加密方法
- 搜索
    + 直接搜索参数
    + pwd=
    + pwd =
    + pwd:
    + pwd :
    + 密码框地方右键 检查 查看 id name type
    + 找到加密的地方(重点)
- 调试
- 找出所有的加密代码
    + 从最后一步开始写起，缺啥找啥
    + 如果找的是函数的话 search 要带上 function xxx
    + 如果看到加密的地方有个类，并且之后是用 prototype 把方法加在原生对象上的话，要把
    + 所有加在原生对象上的方法都找出来
    + 函数找多了没关系，只要不报错不会影响结果，但是不能找少了
    

## 案例
### [Python爬虫进阶必备 | X天下与XX二手房密码加密分析](https://mp.weixin.qq.com/s/R_9W2TKcMzKK46nOHystQw)

房天下url: https://sz.fang.com/


登录页面 - 输入错误密码 - XHR - 找到登录url  - 查看加密字段

![login](login.png)
(上图 From Data 处就是登录信息以表单数据的形式提交给服务器)

> 请求体

请求体一般承载的内容是 POST 请求中的表单数据，而对于 GET 请求，请求体则为空。

登录之前，我们填写了用户名和密码信息，提交时这些内容就会以表单数据的形式提交给服务器，此时需要注意 Request Headers 中指定 Content-Type 为 application/x-www-form-urlencoded。只有设置 Content-Type 为 application/x-www-form-urlencoded，才会以表单数据的形式提交。另外，我们也可以将 Content-Type 设置为 application/json 来提交 JSON 数据，或者设置为 multipart/form-data 来上传文件。

Content-Type|提交数据的方式
---|:--:
application/x-www-form-urlencoded|表单数据
multipart/form-data|表单文件上传
application/json|序列化 JSON 数据
text/xml|XML 数据

在爬虫中，如果要构造 POST 请求，需要使用正确的 Content-Type，并了解各种请求库的各个参数设置时使用的是哪种 Content-Type，不然可能会导致 POST 提交后无法正常响应。


### 构造 POST 请求

原请求体
```
uid: 13480975957
pwd: 01820ee00c589c1125523ce7f05cdebf96c378735362b4eed47aa63ee4d80918630041762ce3326938be98984d3097bd0b40b44bc7a3bc98b47cdf9ae6ee7746caabc1c197c467952140ae28adb8d70a17c4a97b7b84cd0fd32f1e1bb5ea1aa4de0b06e3a46e93841f5222b9589c72c9632e283f0049854e3c768d03fbf7d4ee
Service: soufun-passport-web
AutoLogin: 1
```

- uid 是手机号
- pwd 加密字段


### 探索寻找加密逻辑

在 Search 框直接通过检索pwd:定位加密位置 文件 loginbypassword.js

右击文件 - open in source pannel - 根据检索结果给对应的位置(239行)打上断点，并把划线的代码复制出来备用。
`pwd: encryptedString(key_to_encode, that.password.val()),`

重新发起请求，发现这些参数大概是下面这样的：

```
// that.password.val() 输入的密码：11111111111
// encryptedString 是加密方法
encryptedString(key_to_encode, that.password.val())
```

根据上面分析就少了一个 key_to_encode，在 Search 框直接检索 key_to_encode 可以找到下面的结果

![search](search.png)

很明显的 RSA 加密，将这块js代码复制出来

In [ ]:
# js文件内容第一部分
setMaxDigits(129);
var key_to_encode = new RSAKeyPair("010001", "", "978C0A92D2173439707498F0944AA476B1B62595877DD6FA87F6E2AC6DCB3D0BF0B82857439C99B5091192BC134889DFF60C562EC54EFBA4FF2F9D55ADBCCEA4A2FBA80CB398ED501280A007C83AF30C3D1A142D6133C63012B90AB26AC60C898FB66EDC3192C3EC4FF66925A64003B72496099F4F09A9FB72A2CF9E4D770C41");

这里 setMaxDigits() 未知，在 Search 框直接检索找到 RSA.min.js，里面还有 encryptedString 函数，将文件中所有内容复制出来

In [ ]:
# js文件内容第二部分
function setMaxDigits(n) {
    maxDigits = n;
    ZERO_ARRAY = new Array(maxDigits);
    for (var t = 0; t < ZERO_ARRAY.length; t++)
        ZERO_ARRAY[t] = 0;
    bigZero = new BigInt;
    bigOne = new BigInt;
    bigOne.digits[0] = 1
}
function BigInt(n) {
    this.digits = typeof n == "boolean" && n == !0 ? null : ZERO_ARRAY.slice(0);
    this.isNeg = !1
}
function biFromDecimal(n) {
    for (var u = n.charAt(0) == "-", t = u ? 1 : 0, i, f, r; t < n.length && n.charAt(t) == "0"; )
        ++t;
    if (t == n.length)
        i = new BigInt;
    else {
        for (f = n.length - t,
        r = f % dpl10,
        r == 0 && (r = dpl10),
        i = biFromNumber(Number(n.substr(t, r))),
        t += r; t < n.length; )
            i = biAdd(biMultiply(i, lr10), biFromNumber(Number(n.substr(t, dpl10)))),
            t += dpl10;
        i.isNeg = u
    }
    return i
}
function biCopy(n) {
    var t = new BigInt(!0);
    return t.digits = n.digits.slice(0),
    t.isNeg = n.isNeg,
    t
}
function biFromNumber(n) {
    var t = new BigInt, i;
    for (t.isNeg = n < 0,
    n = Math.abs(n),
    i = 0; n > 0; )
        t.digits[i++] = n & maxDigitVal,
        n = Math.floor(n / biRadix);
    return t
}
function reverseStr(n) {
    for (var i = "", t = n.length - 1; t > -1; --t)
        i += n.charAt(t);
    return i
}
function biToString(n, t) {
    var r = new BigInt, i, u;
    for (r.digits[0] = t,
    i = biDivideModulo(n, r),
    u = hexatrigesimalToChar[i[1].digits[0]]; biCompare(i[0], bigZero) == 1; )
        i = biDivideModulo(i[0], r),
        digit = i[1].digits[0],
        u += hexatrigesimalToChar[i[1].digits[0]];
    return (n.isNeg ? "-" : "") + reverseStr(u)
}
function biToDecimal(n) {
    var i = new BigInt, t, r;
    for (i.digits[0] = 10,
    t = biDivideModulo(n, i),
    r = String(t[1].digits[0]); biCompare(t[0], bigZero) == 1; )
        t = biDivideModulo(t[0], i),
        r += String(t[1].digits[0]);
    return (n.isNeg ? "-" : "") + reverseStr(r)
}
function digitToHex(n) {
    var t = "";
    for (i = 0; i < 4; ++i)
        t += hexToChar[n & 15],
        n >>>= 4;
    return reverseStr(t)
}
function biToHex(n) {
    for (var i = "", r = biHighIndex(n), t = biHighIndex(n); t > -1; --t)
        i += digitToHex(n.digits[t]);
    return i
}
function charToHex(n) {
    var t = 48
      , u = t + 9
      , i = 97
      , f = i + 25
      , r = 65;
    return n >= t && n <= u ? n - t : n >= r && n <= 90 ? 10 + n - r : n >= i && n <= f ? 10 + n - i : 0
}
function hexToDigit(n) {
    for (var t = 0, r = Math.min(n.length, 4), i = 0; i < r; ++i)
        t <<= 4,
        t |= charToHex(n.charCodeAt(i));
    return t
}
function biFromHex(n) {
    for (var i = new BigInt, u = n.length, t = u, r = 0; t > 0; t -= 4,
    ++r)
        i.digits[r] = hexToDigit(n.substr(Math.max(t - 4, 0), Math.min(t, 4)));
    return i
}
function biFromString(n, t) {
    var f = n.charAt(0) == "-", e = f ? 1 : 0, i = new BigInt, r = new BigInt, u;
    for (r.digits[0] = 1,
    u = n.length - 1; u >= e; u--) {
        var o = n.charCodeAt(u)
          , s = charToHex(o)
          , h = biMultiplyDigit(r, s);
        i = biAdd(i, h);
        r = biMultiplyDigit(r, t)
    }
    return i.isNeg = f,
    i
}
function biDump(n) {
    return (n.isNeg ? "-" : "") + n.digits.join(" ")
}
function biAdd(n, t) {
    var r, u, f, i;
    if (n.isNeg != t.isNeg)
        t.isNeg = !t.isNeg,
        r = biSubtract(n, t),
        t.isNeg = !t.isNeg;
    else {
        for (r = new BigInt,
        u = 0,
        i = 0; i < n.digits.length; ++i)
            f = n.digits[i] + t.digits[i] + u,
            r.digits[i] = f % biRadix,
            u = Number(f >= biRadix);
        r.isNeg = n.isNeg
    }
    return r
}
function biSubtract(n, t) {
    var r, f, u, i;
    if (n.isNeg != t.isNeg)
        t.isNeg = !t.isNeg,
        r = biAdd(n, t),
        t.isNeg = !t.isNeg;
    else {
        for (r = new BigInt,
        u = 0,
        i = 0; i < n.digits.length; ++i)
            f = n.digits[i] - t.digits[i] + u,
            r.digits[i] = f % biRadix,
            r.digits[i] < 0 && (r.digits[i] += biRadix),
            u = 0 - Number(f < 0);
        if (u == -1) {
            for (u = 0,
            i = 0; i < n.digits.length; ++i)
                f = 0 - r.digits[i] + u,
                r.digits[i] = f % biRadix,
                r.digits[i] < 0 && (r.digits[i] += biRadix),
                u = 0 - Number(f < 0);
            r.isNeg = !n.isNeg
        } else
            r.isNeg = n.isNeg
    }
    return r
}
function biHighIndex(n) {
    for (var t = n.digits.length - 1; t > 0 && n.digits[t] == 0; )
        --t;
    return t
}
function biNumBits(n) {
    for (var i = biHighIndex(n), r = n.digits[i], u = (i + 1) * bitsPerDigit, t = u; t > u - bitsPerDigit; --t) {
        if ((r & 32768) != 0)
            break;
        r <<= 1
    }
    return t
}
function biMultiply(n, t) {
    for (var i = new BigInt, u, o = biHighIndex(n), s = biHighIndex(t), e, f, r = 0; r <= s; ++r) {
        for (u = 0,
        f = r,
        j = 0; j <= o; ++j,
        ++f)
            e = i.digits[f] + n.digits[j] * t.digits[r] + u,
            i.digits[f] = e & maxDigitVal,
            u = e >>> biRadixBits;
        i.digits[r + o + 1] = u
    }
    return i.isNeg = n.isNeg != t.isNeg,
    i
}
function biMultiplyDigit(n, t) {
    var u, r, f, i;
    for (result = new BigInt,
    u = biHighIndex(n),
    r = 0,
    i = 0; i <= u; ++i)
        f = result.digits[i] + n.digits[i] * t + r,
        result.digits[i] = f & maxDigitVal,
        r = f >>> biRadixBits;
    return result.digits[1 + u] = r,
    result
}
function arrayCopy(n, t, i, r, u) {
    for (var o = Math.min(t + u, n.length), f = t, e = r; f < o; ++f,
    ++e)
        i[e] = n[f]
}
function biShiftLeft(n, t) {
    var e = Math.floor(t / bitsPerDigit), i = new BigInt, u, o, r, f;
    for (arrayCopy(n.digits, 0, i.digits, e, i.digits.length - e),
    u = t % bitsPerDigit,
    o = bitsPerDigit - u,
    r = i.digits.length - 1,
    f = r - 1; r > 0; --r,
    --f)
        i.digits[r] = i.digits[r] << u & maxDigitVal | (i.digits[f] & highBitMasks[u]) >>> o;
    return i.digits[0] = i.digits[r] << u & maxDigitVal,
    i.isNeg = n.isNeg,
    i
}
function biShiftRight(n, t) {
    var e = Math.floor(t / bitsPerDigit), i = new BigInt, u, o, r, f;
    for (arrayCopy(n.digits, e, i.digits, 0, n.digits.length - e),
    u = t % bitsPerDigit,
    o = bitsPerDigit - u,
    r = 0,
    f = r + 1; r < i.digits.length - 1; ++r,
    ++f)
        i.digits[r] = i.digits[r] >>> u | (i.digits[f] & lowBitMasks[u]) << o;
    return i.digits[i.digits.length - 1] >>>= u,
    i.isNeg = n.isNeg,
    i
}
function biMultiplyByRadixPower(n, t) {
    var i = new BigInt;
    return arrayCopy(n.digits, 0, i.digits, t, i.digits.length - t),
    i
}
function biDivideByRadixPower(n, t) {
    var i = new BigInt;
    return arrayCopy(n.digits, t, i.digits, 0, i.digits.length - t),
    i
}
function biModuloByRadixPower(n, t) {
    var i = new BigInt;
    return arrayCopy(n.digits, 0, i.digits, 0, t),
    i
}
function biCompare(n, t) {
    if (n.isNeg != t.isNeg)
        return 1 - 2 * Number(n.isNeg);
    for (var i = n.digits.length - 1; i >= 0; --i)
        if (n.digits[i] != t.digits[i])
            return n.isNeg ? 1 - 2 * Number(n.digits[i] > t.digits[i]) : 1 - 2 * Number(n.digits[i] < t.digits[i]);
    return 0
}
function biDivideModulo(n, t) {
    var a = biNumBits(n), s = biNumBits(t), v = t.isNeg, r, i, u, e, h, o, f, y, p;
    if (a < s)
        return n.isNeg ? (r = biCopy(bigOne),
        r.isNeg = !t.isNeg,
        n.isNeg = !1,
        t.isNeg = !1,
        i = biSubtract(t, n),
        n.isNeg = !0,
        t.isNeg = v) : (r = new BigInt,
        i = biCopy(n)),
        [r, i];
    for (r = new BigInt,
    i = n,
    u = Math.ceil(s / bitsPerDigit) - 1,
    e = 0; t.digits[u] < biHalfRadix; )
        t = biShiftLeft(t, 1),
        ++e,
        ++s,
        u = Math.ceil(s / bitsPerDigit) - 1;
    for (i = biShiftLeft(i, e),
    a += e,
    h = Math.ceil(a / bitsPerDigit) - 1,
    o = biMultiplyByRadixPower(t, h - u); biCompare(i, o) != -1; )
        ++r.digits[h - u],
        i = biSubtract(i, o);
    for (f = h; f > u; --f) {
        var c = f >= i.digits.length ? 0 : i.digits[f]
          , w = f - 1 >= i.digits.length ? 0 : i.digits[f - 1]
          , b = f - 2 >= i.digits.length ? 0 : i.digits[f - 2]
          , l = u >= t.digits.length ? 0 : t.digits[u]
          , k = u - 1 >= t.digits.length ? 0 : t.digits[u - 1];
        for (r.digits[f - u - 1] = c == l ? maxDigitVal : Math.floor((c * biRadix + w) / l),
        y = r.digits[f - u - 1] * (l * biRadix + k),
        p = c * biRadixSquared + (w * biRadix + b); y > p; )
            --r.digits[f - u - 1],
            y = r.digits[f - u - 1] * (l * biRadix | k),
            p = c * biRadix * biRadix + (w * biRadix + b);
        o = biMultiplyByRadixPower(t, f - u - 1);
        i = biSubtract(i, biMultiplyDigit(o, r.digits[f - u - 1]));
        i.isNeg && (i = biAdd(i, o),
        --r.digits[f - u - 1])
    }
    return i = biShiftRight(i, e),
    r.isNeg = n.isNeg != v,
    n.isNeg && (r = v ? biAdd(r, bigOne) : biSubtract(r, bigOne),
    t = biShiftRight(t, e),
    i = biSubtract(t, i)),
    i.digits[0] == 0 && biHighIndex(i) == 0 && (i.isNeg = !1),
    [r, i]
}
function biDivide(n, t) {
    return biDivideModulo(n, t)[0]
}
function biModulo(n, t) {
    return biDivideModulo(n, t)[1]
}
function biMultiplyMod(n, t, i) {
    return biModulo(biMultiply(n, t), i)
}
function biPow(n, t) {
    for (var r = bigOne, i = n; ; ) {
        if ((t & 1) != 0 && (r = biMultiply(r, i)),
        t >>= 1,
        t == 0)
            break;
        i = biMultiply(i, i)
    }
    return r
}
function biPowMod(n, t, i) {
    for (var f = bigOne, u = n, r = t; ; ) {
        if ((r.digits[0] & 1) != 0 && (f = biMultiplyMod(f, u, i)),
        r = biShiftRight(r, 1),
        r.digits[0] == 0 && biHighIndex(r) == 0)
            break;
        u = biMultiplyMod(u, u, i)
    }
    return f
}
function BarrettMu(n) {
    this.modulus = biCopy(n);
    this.k = biHighIndex(this.modulus) + 1;
    var t = new BigInt;
    t.digits[2 * this.k] = 1;
    this.mu = biDivide(t, this.modulus);
    this.bkplus1 = new BigInt;
    this.bkplus1.digits[this.k + 1] = 1;
    this.modulo = BarrettMu_modulo;
    this.multiplyMod = BarrettMu_multiplyMod;
    this.powMod = BarrettMu_powMod
}
function BarrettMu_modulo(n) {
    var r = biDivideByRadixPower(n, this.k - 1), u = biMultiply(r, this.mu), f = biDivideByRadixPower(u, this.k + 1), e = biModuloByRadixPower(n, this.k + 1), o = biMultiply(f, this.modulus), s = biModuloByRadixPower(o, this.k + 1), t = biSubtract(e, s), i;
    for (t.isNeg && (t = biAdd(t, this.bkplus1)),
    i = biCompare(t, this.modulus) >= 0; i; )
        t = biSubtract(t, this.modulus),
        i = biCompare(t, this.modulus) >= 0;
    return t
}
function BarrettMu_multiplyMod(n, t) {
    var i = biMultiply(n, t);
    return this.modulo(i)
}
function BarrettMu_powMod(n, t) {
    var u = new BigInt, r, i;
    for (u.digits[0] = 1,
    r = n,
    i = t; ; ) {
        if ((i.digits[0] & 1) != 0 && (u = this.multiplyMod(u, r)),
        i = biShiftRight(i, 1),
        i.digits[0] == 0 && biHighIndex(i) == 0)
            break;
        r = this.multiplyMod(r, r)
    }
    return u
}
function RSAKeyPair(n, t, i) {
    this.e = biFromHex(n);
    this.d = biFromHex(t);
    this.m = biFromHex(i);
    this.digitSize = 2 * biHighIndex(this.m) + 2;
    this.chunkSize = this.digitSize - 11;
    this.radix = 16;
    this.barrett = new BarrettMu(this.m)
}
function twoDigit(n) {
    return (n < 10 ? "0" : "") + String(n)
}
function encryptedString(n, t) {
    var e, o, s, h, c, i, f, u, v, l, y;
    if (n.chunkSize > n.digitSize - 11)
        return "Error";
    for (var a = [], p = t.length, r = 0; r < p; )
        a[r] = t.charCodeAt(r),
        r++;
    for (e = a.length,
    o = "",
    r = 0; r < e; r += n.chunkSize) {
        for (c = new BigInt,
        s = 0,
        f = r + n.chunkSize > e ? e % n.chunkSize : n.chunkSize,
        u = [],
        i = 0; i < f; i++)
            u[i] = a[r + f - 1 - i];
        for (u[f] = 0,
        v = Math.max(8, n.digitSize - 3 - f),
        i = 0; i < v; i++)
            u[f + 1 + i] = Math.floor(Math.random() * 254) + 1;
        for (u[n.digitSize - 2] = 2,
        u[n.digitSize - 1] = 0,
        h = 0; h < n.digitSize; ++s)
            c.digits[s] = u[h++],
            c.digits[s] += u[h++] << 8;
        l = n.barrett.powMod(c, n.e);
        y = n.radix == 16 ? biToHex(l) : biToString(l, n.radix);
        o += y + " "
    }
    return o.substring(0, o.length - 1)
}
function decryptedString(n, t) {
    for (var e = t.split(" "), i = "", r, u, o, f = 0; f < e.length; ++f)
        for (o = n.radix == 16 ? biFromHex(e[f]) : biFromString(e[f], n.radix),
        u = n.barrett.powMod(o, n.d),
        r = 0; r <= biHighIndex(u); ++r)
            i += String.fromCharCode(u.digits[r] & 255, u.digits[r] >> 8);
    return i.charCodeAt(i.length - 1) == 0 && (i = i.substring(0, i.length - 1)),
    i
}
var biRadixBase = 2, biRadixBits = 16, bitsPerDigit = biRadixBits, biRadix = 65536, biHalfRadix = biRadix >>> 1, biRadixSquared = biRadix * biRadix, maxDigitVal = biRadix - 1, maxInteger = 9999999999999998, maxDigits, ZERO_ARRAY, bigZero, bigOne, dpl10, lr10, hexatrigesimalToChar, hexToChar, highBitMasks, lowBitMasks;
setMaxDigits(20);
dpl10 = 15;
lr10 = biFromNumber(1e15);
hexatrigesimalToChar = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n", "o", "p", "q", "r", "s", "t", "u", "v", "w", "x", "y", "z"];
hexToChar = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "a", "b", "c", "d", "e", "f"];
highBitMasks = [0, 32768, 49152, 57344, 61440, 63488, 64512, 65024, 65280, 65408, 65472, 65504, 65520, 65528, 65532, 65534, 65535];
lowBitMasks = [0, 1, 3, 7, 15, 31, 63, 127, 255, 511, 1023, 2047, 4095, 8191, 16383, 32767, 65535];
setMaxDigits(129);

### js 文件构造

js文件内容第二部分 + js文件内容第一部分 = 加密逻辑

js文件最后加上构造函数，用来返回加密字段 PWD 

In [ ]:
function getEncryptedData(pwd) {
    var pwd = pwd;
    var key_to_encode = new RSAKeyPair("010001", "", "978C0A92D2173439707498F0944AA476B1B62595877DD6FA87F6E2AC6DCB3D0BF0B82857439C99B5091192BC134889DFF60C562EC54EFBA4FF2F9D55ADBCCEA4A2FBA80CB398ED501280A007C83AF30C3D1A142D6133C63012B90AB26AC60C898FB66EDC3192C3EC4FF66925A64003B72496099F4F09A9FB72A2CF9E4D770C41");
    return encryptedString(key_to_encode, pwd);
}

### 构造登录 request

将原参数复制出来，request 的框架

In [ ]:
import requests

url = 'https://passport.fang.com/login.api'
# 请求头
headers = {
    'Cookie' : 'city=sz; global_cookie=t6mlw9eosdkpqm2a201ucolqk1tk0ghyzs4; __jsluid_s=37d7da7754ddb989b8644ceea4fa7fa3; token=63012c62083741c3bcd0e30cd6bdb328; unique_cookie=U_t6mlw9eosdkpqm2a201ucolqk1tk0ghyzs4*28',
    'Referer' : 'https://passport.fang.com/?backurl=https%3A%2F%2Fsz.fang.com%2F',
    'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.100 Safari/537.36',
}
# 请求体
from_data = {
    'uid': '452070961@qq.com',
    'pwd': 3e34671dab0fd303c322b2c42b11b3000caae84ea3124c3f4ef540e3158361fec693362f6cba4b638fbcf31ecd526f40d47981d389342fbfe9ff46884f419730da7433d2719ef04fe9095600feb8895b1a9ba095d5ea80aa961f9bfe0882e354fec32c68be98b6c20c9352f7b4c7d34c6ec24c74a827ca89f01d03c049ed7696,
    'Service': 'soufun-passport-web',
    'AutoLogin': 0,
}

response = requests.post(url,data=from_data,headers=headers)
print(response.content)

### python 中调用 js

In [1]:
import execjs

# Init environment 环境初始化
node = execjs.get()

# Params 参数
userid = '13480975957'
pwd = '11111111111'

# Compile javascript 编译js文件
file = 'fangtianxia.js'
ctx = node.compile(open(file).read())

# Get params 获取参数
js = 'getEncryptedData("{}")'.format(pwd)
print(js)
enpwd = ctx.eval(js)
print(enpwd)

getEncryptedData("11111111111")
948cee869d1ae5c3f394dc2460b212180a51b278cefaae8b7149cf43fbb28d759815a227ae210324177331c082ede36d443873d5b969ded95d45e2c4841ba75de83a33c3111470a2751112b7045afb573c87471c0bb068844a558d4c56b9fc0ed184020393898fc94f9185c5b61c134cb6bdc6be33a10e6372f63e37cfb6872d


In [8]:
import requests

url = 'https://passport.fang.com/login.api'
headers = {
    'Cookie' : 'city=sz; global_cookie=t6mlw9eosdkpqm2a201ucolqk1tk0ghyzs4; __jsluid_s=37d7da7754ddb989b8644ceea4fa7fa3; token=63012c62083741c3bcd0e30cd6bdb328; unique_cookie=U_t6mlw9eosdkpqm2a201ucolqk1tk0ghyzs4*28',
    'Referer' : 'https://passport.fang.com/?backurl=https%3A%2F%2Fsz.fang.com%2F',
    'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.100 Safari/537.36',
}

from_data = {
    'uid':userid,
    'pwd': enpwd,
    'Service':'soufun-passport-web',
    'AutoLogin':0,
}

response = requests.post(url,data=from_data,headers=headers)
print(response.content)

b'{"Message":"Error","Tip":"\xe8\xaf\xa5\xe7\x94\xa8\xe6\x88\xb7\xe6\x9c\xaa\xe6\xb3\xa8\xe5\x86\x8c","UserID":0,"UserName":null,"UrlReferrer":null,"BackUrl":null,"LoginErrorCount":0,"Ip":"219.133.101.42","IsIpCanVisit":"true","IsVerificationByMobilePhone":0,"PToken":null,"NickName":null,"UserId":0,"Avatar":null}'


In [22]:
string = '\xe8\xaf\xa5\xe7\x94\xa8\xe6\x88\xb7\xe6\x9c\xaa\xe6\xb3\xa8\xe5\x86\x8c'
ss = string.encode('raw_unicode_escape')
print(ss.decode()) 

该用户未注册


## 完整脚本，用正确账号登录(可运行)

In [32]:
import execjs
import requests

# Init environment
node = execjs.get()

# Params
userid = '13106141814'
pwd = 'Jamie8449671'

# Compile javascript
file = 'fangtianxia.js'
ctx = node.compile(open(file).read())

# Get params
js = 'getEncryptedData("{}")'.format(pwd)
enpwd = ctx.eval(js)

url = 'https://passport.fang.com/login.api'
headers = {
    'Cookie' : 'city=sz; global_cookie=t6mlw9eosdkpqm2a201ucolqk1tk0ghyzs4; __jsluid_s=37d7da7754ddb989b8644ceea4fa7fa3; token=63012c62083741c3bcd0e30cd6bdb328; unique_cookie=U_t6mlw9eosdkpqm2a201ucolqk1tk0ghyzs4*28',
    'Referer' : 'https://passport.fang.com/?backurl=https%3A%2F%2Fsz.fang.com%2F',
    'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.100 Safari/537.36',
}

from_data = {
    'uid':userid,
    'pwd': enpwd,
    'Service':'soufun-passport-web',
    'AutoLogin':0,
}

response = requests.post(url,data=from_data,headers=headers)
print(response.content)

user_url = 'https://my.fang.com'
r = requests.get(user_url)
print(r.content)

b'{"Message":"Success","Tip":"","UserID":113984099,"UserName":"xujiu0303","UrlReferrer":null,"BackUrl":"","LoginErrorCount":0,"Ip":"219.133.101.42","IsIpCanVisit":"true","IsVerificationByMobilePhone":1,"PToken":"","NickName":"","UserId":0,"Avatar":""}'
b"<script>window.top.location='https://passport.fang.com/?backurl=http%3a%2f%2fmy.fang.com%2f'</script>"


上面先用 post方法成功登录网站，并get 方法去请求个人信息页面，结果并没有获取到个人信息页面。

在 requests 中，如果直接利用 get 或 post 等方法的确可以做到模拟网页的请求，但是这实际上是相当于不同的会话，也就是说相当于你用了两个浏览器打开了不同的页面。

设想这样一个场景，第一个请求利用 post 方法登录了某个网站，第二次想获取成功登录后的自己的个人信息，你又用了一次 get 方法去请求个人信息页面。实际上，这相当于打开了两个浏览器，是两个完全不相关的会话，能成功获取个人信息吗？那当然不能。

有小伙伴可能说了，我在两次请求时设置一样的 cookies 不就行了？可以，但这样做起来显得很烦琐，我们有更简单的解决方法。

其实解决这个问题的主要方法就是维持同一个会话，也就是相当于打开一个新的浏览器选项卡而不是新开一个浏览器。但是我又不想每次设置 cookies，那该怎么办呢？这时候就有了新的利器 ——Session 对象。

## 利用 session，用正确账号登录，并获取个人信息页(可运行)

In [34]:
import execjs
import requests

# Init environment
node = execjs.get()

# Params
userid = '13106141814'
pwd = 'Jamie8449671'

# Compile javascript
file = 'fangtianxia.js'
ctx = node.compile(open(file).read())

# Get params
js = 'getEncryptedData("{}")'.format(pwd)
enpwd = ctx.eval(js)

url = 'https://passport.fang.com/login.api'
headers = {
    'Cookie' : 'city=sz; global_cookie=t6mlw9eosdkpqm2a201ucolqk1tk0ghyzs4; __jsluid_s=37d7da7754ddb989b8644ceea4fa7fa3; token=63012c62083741c3bcd0e30cd6bdb328; unique_cookie=U_t6mlw9eosdkpqm2a201ucolqk1tk0ghyzs4*28',
    'Referer' : 'https://passport.fang.com/?backurl=https%3A%2F%2Fsz.fang.com%2F',
    'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.100 Safari/537.36',
}

from_data = {
    'uid':userid,
    'pwd': enpwd,
    'Service':'soufun-passport-web',
    'AutoLogin':0,
}


s = requests.Session()
response = s.post(url,data=from_data,headers=headers)
print(response.content)

user_url = 'https://my.fang.com'
r = s.get(user_url)
print(r.content)

b'{"Message":"Success","Tip":"","UserID":113984099,"UserName":"xujiu0303","UrlReferrer":null,"BackUrl":"","LoginErrorCount":0,"Ip":"219.133.101.42","IsIpCanVisit":"true","IsVerificationByMobilePhone":1,"PToken":"","NickName":"","UserId":0,"Avatar":""}'
b'<!DOCTYPE html>\r\n<html>\r\n<head>\r\n    <meta http-equiv="Content-Type" content="text/html; charset=gb2312" />\r\n    <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1" />\r\n    <title>\xce\xd2\xb5\xc4\xb7\xbf\xcc\xec\xcf\xc2</title>\r\n    <link href="//img.soufunimg.com/newhousedata/images/My/ResourceNew/Css/common.css?Version=1.0.0.26" rel="stylesheet" />\r\n\r\n    <link href="//img.soufunimg.com/newhousedata/images/My/ResourceNew/Css/css2018.css?Version=1.1.6" rel="stylesheet" />\r\n    <script type="text/javascript" src="//img.soufunimg.com/newhousedata/images/My/ResourceNew/Script/jquery.min.js?Version=1.0.0.26"></script>\r\n    <script src="//img.soufunimg.com/newhousedata/images/My/ResourceNew/Script/dhjs.js?Ver

上面利用 session 成功登录了网站并获取到个人信息页，但是这个源码编码了

我们用'utf-8'解码报错了，发现响应头里面标识了编码类型。我们直接用这个解码，得到下面结果。
```
content-encoding: gzip
content-type: text/html; charset=gb2312
```

In [41]:
str(r.content, encoding = "gb2312")  

'<!DOCTYPE html>\r\n<html>\r\n<head>\r\n    <meta http-equiv="Content-Type" content="text/html; charset=gb2312" />\r\n    <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1" />\r\n    <title>我的房天下</title>\r\n    <link href="//img.soufunimg.com/newhousedata/images/My/ResourceNew/Css/common.css?Version=1.0.0.26" rel="stylesheet" />\r\n\r\n    <link href="//img.soufunimg.com/newhousedata/images/My/ResourceNew/Css/css2018.css?Version=1.1.6" rel="stylesheet" />\r\n    <script type="text/javascript" src="//img.soufunimg.com/newhousedata/images/My/ResourceNew/Script/jquery.min.js?Version=1.0.0.26"></script>\r\n    <script src="//img.soufunimg.com/newhousedata/images/My/ResourceNew/Script/dhjs.js?Version=1.0.0.26"></script>\r\n    <script src="//img.soufunimg.com/newhousedata/images/My/ResourceNew/Script/Common.js?Version=1.0.0.26"></script>\r\n    <script src="//img.soufunimg.com/newhousedata/images/My/ResourceNew/Script/validation.js?v=201805"></script>\r\n    <script src="//img.so